![alt text](images\HDAT9500Banner.PNG)
<br>

# Assessment 7

## 1.1. Bisecting K-Means

K-Means often results in clusters of widely different sizes. In this assessment you are asked to implement an extension to k-means called bisecting k-means. The algorithm proceeds as follows:

1. Pick a cluster to split (for this exercise, always pick the largest one)
2. Find 2 sub-clusters using the basic k-means algorithm (bisecting step)
3. Repeat step 2 for ITER times (for this exercise, set ITER=20) and take the split that minimizes the inertia
4. Repeat steps 1, 2 and 3 until the desired number of clusters is reached

## 1.2. Tasks

1. Implement the bisecting k-means algorithm.
2. Apply bisecting k-means to the Breast Cancer Wisconsin (Diagnostic) Data Set using the first 10 numerical features in the dataset as feature vectors (as was done in the practical exercise "Exercise 7 - PCA"). Remember to scale the data to have zero mean and unit variance before clustering. Run the algorithm three different times so that the data are divided into: (a) 2 clusters, (b) 5 clusters, (c) 10 clusters.
3. Compare the number of observations in each cluster between basic k-means and bisecting k-means when the data are divided into 10 clusters.

## 1.2. Aims:

1. Gain a better understanding of k-means and clustering algorithms in general.

## 1.3. Tips:

You are allowed to use any function that was used in the practical exercises.

In [3]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

def bisecting_kmeans(df, n_clusters, n_init=20):
    '''Implements the bisecting kmeans algorithm.
    
    Args:
        df: pandas dataframe to which the bisecting kmeans algorithms is applied.
            Has observations as rows and features as columns.
        num_clusters: int. The desired number of clusters.
        n_init: int, default: 20. Number of time the k-means algorithm will be run with different centroid seeds. 
            The final results will be the best output of n_init consecutive runs in terms of inertia.

    Returns:
        df: pandas dataframe; same as input df with an additional column 'CLUSTER_LABEL'
    '''
        
    # add column with cluster labels, set all to 0 initially
    df['CLUSTER_LABEL'] = 0
    
    # kmeans instance
    kmeans = KMeans(n_clusters=2, n_init=n_init)
    
    # iteratively bisect largest cluster
    for _ in range(n_clusters-1):
        # indices of largest cluster. Use .mode().iloc[0] since .mode() might return multiple values
        # in case of two equally large clusters
        largest_idx = (df['CLUSTER_LABEL'] == df['CLUSTER_LABEL'].mode().iloc[0])
        # bisect largest cluster and get labels (0 and 1)
        largest_labels = kmeans.fit(df.loc[largest_idx, df.columns!='CLUSTER_LABEL']).labels_
        # update labels of largest cluster
        # add 1 to max label so that the new labels are different from any previous ones
        df.loc[largest_idx, 'CLUSTER_LABEL'] = largest_labels + df['CLUSTER_LABEL'].max() + 1
    
    # optionally tidy up labels so that they go from 0 to (num_clusters-1)
    labels = df['CLUSTER_LABEL'].unique()
    dict_labels = {labels[i] : i for i in range(len(labels))}
    df['CLUSTER_LABEL'] = df['CLUSTER_LABEL'].map(dict_labels)
    
    # return result
    return df

In [4]:
# load Breast Cancer Wisconsin (Diagnostic) Data Set using the first 10 numerical features
bcw = pd.read_csv('../data/breast-cancer-wisconsin-data/data.csv', sep=',')
X = bcw[bcw.columns[2:12]]

# scale data
X = (X - X.mean(axis=0)) / X.std(axis=0)

print('bisecting kmeans, 2 clusters')
X_clustered = bisecting_kmeans(X, n_clusters=2, n_init=20)
print(X_clustered['CLUSTER_LABEL'].value_counts())
print('----------------------------')

print('bisecting kmeans, 5 clusters')
X_clustered = bisecting_kmeans(X, n_clusters=5, n_init=20)
print(X_clustered['CLUSTER_LABEL'].value_counts())
print('----------------------------')

print('bisecting kmeans, 10 clusters')
X_clustered = bisecting_kmeans(X, n_clusters=10, n_init=20)
print(X_clustered['CLUSTER_LABEL'].value_counts())
print('----------------------------')

print('basic kmeans, 10 clusters')
kmeans = KMeans(n_clusters=10, n_init=20)
X['CLUSTER_LABEL'] = kmeans.fit(X).labels_
print(X['CLUSTER_LABEL'].value_counts())
print('----------------------------')


bisecting kmeans, 2 clusters
1    400
0    169
Name: CLUSTER_LABEL, dtype: int64
----------------------------
bisecting kmeans, 5 clusters
3    158
4    153
1    101
2     89
0     68
Name: CLUSTER_LABEL, dtype: int64
----------------------------
bisecting kmeans, 10 clusters
4    80
7    78
1    71
5    69
0    68
3    47
6    46
8    42
9    38
2    30
Name: CLUSTER_LABEL, dtype: int64
----------------------------
basic kmeans, 10 clusters
3    91
7    78
9    77
0    69
8    62
2    49
5    46
4    43
6    38
1    16
Name: CLUSTER_LABEL, dtype: int64
----------------------------


This illustrates that bisecting kmeans is less likely to generate very large or very small clusters compared with basic kmeans.